In [19]:
# Importando módulos internos
import geocov19_functions_db as fdb
import geocov19_functions_tweets as ftweets
import geocov19_functions_text as ftext

# Importando módulos externos
import pandas as pd

The history saving thread hit an unexpected error (OperationalError('no such table: history')).History will not be written to the database.


In [20]:
# Criando estrutura do banco de dados
from pymongo import MongoClient

# Conexão com o servidor do MongoDB
clienteDB = MongoClient('localhost', 27017)

# Conexão com a base de dados do mongoDB
banco = clienteDB.SpedDB

# Coleção onde serão inseridos os dados
collection = banco.tweets_brasil

In [21]:
from google_trans_new import google_translator  

translator = google_translator()  

def translate_tweet_text_new(tweet):
    
    translated = ''
        
    text = tweet['text']   
    from_lang = tweet['lang']
    
    # Verificando se o texto do tweet já está escrito na língua inglesa
    if (from_lang == 'en') or (from_lang == 'en-US'):
        translated = text
    else:
        translated = translator.translate(text, lang_tgt='en')

    return translated

In [22]:
import sys

# Ferramenta SenticNet
def calculate_polarities(tweets):
        
    for tweet in tweets:        
        try:                 
            # Traduzindo o texto para o inglês
            # translated_text = translate_tweet_text(tweet)
            translated_text = translate_tweet_text_new(tweet)

            # Realizando a limpeza da frase
            words = ftext.process_tweet(translated_text, 'english')

            # Efetuando o cálculo do score de sentimento para o texto
            polarity = calculate_words_polarities(words)

            # Recuperando o score e adicionando em uma nova célula do dataframe
            collection.update_one({"tweet_id": tweet['tweet_id']},{'$set':{"polarity": polarity, "senticnet_reprocessed": True}})  
        
        except Exception as e:
            e = sys.exc_info()[1]
            print ("Unexpected error:", e)
              

In [23]:
from senticnet.senticnet import SenticNet

def calculate_words_polarities(words):
    
    if(len(words) > 0):
        min_words_count = len(words)/2
        #not_found_words_count = 0
        found_words = 0
        total_polarity_value = 0
        language_code = 'en'

        for word in words:
            word = word.lower()
            try:
                sn = SenticNet('en')
                polarity_str = sn.polarity_intense(word)
                #print(word + ': ' + polarity_str)
                polarity_value = float(polarity_str)      
                total_polarity_value = total_polarity_value + polarity_value
                found_words = found_words + 1
            except KeyError:
                e = sys.exc_info()[1]
                #print ("Unexpected error:", e)
                #not_found_words_count = not_found_words_count + 1

        # O cálculo será considerado somente se ao menos metade das palavras forem encontradas na base de conhecimento do SenticNet
        #if not_found_words_count > min_words_count:
        if found_words < min_words_count:
            return None
        else:
            #print(found_words)
            polarity = total_polarity_value / found_words
            #print(polarity)
            return polarity
    else:
        return None

In [25]:
state = 'Tocantins'
results = collection.find({'state':state, "senticnet_processed": True, "senticnet_reprocessed": {'$eq':None}, 'polarity': {'$ne':None}}, {'tweet_id':1,'lang':1,'text':1,'_id': 0});
calculate_polarities(results)